# Importing the Libraries

In [ ]:
import torch
from IPython.display import Image
import shutil
import os
from random import choice

# Removing the dataset and cloned yolo files if they exist already

In [ ]:
try:
    shutil.rmtree("/kaggle/working/dataset")
    shutil.rmtree("/kaggle/working/yolov5")
except:
    pass

# Cloning YoloV5 git Repo

In [ ]:
!git clone https://github.com/ultralytics/yolov5

In [ ]:
%cd yolov5/

# Installing YoloV5 requirements files

In [ ]:
!pip install -r requirements.txt

# Preparing the traffic signs dataset for training and validation

In [ ]:
# arrays to store file names
imgs = []
xmls = []

# setup directory names
train_path = "/kaggle/working/dataset/images/train"
val_path = "/kaggle/working/dataset/images/val"
crs_path = "/kaggle/input/traffic-signs-dataset-in-yolo-format/ts/ts"

train_ratio = 0.8
val_ratio = 0.2

# total count of imgs
total_img_count = len(os.listdir(crs_path))/2

# sorting count of imgs
for (dirname,dirs, files) in os.walk(crs_path):
#     print(files)
    for filename in files:
#         print("hello")
        if filename.endswith(".txt"):
            xmls.append(filename)
        else:
            imgs.append(filename)

# counting range for cycles
count_for_train = int(len(imgs)*train_ratio)
count_for_val = int(len(imgs)*val_ratio)
print("training images are : ",count_for_train)
print("validation images are : ",count_for_val)

In [ ]:
len(imgs)

In [ ]:
train_image_path = "/kaggle/working/dataset/images/train"
train_label_path = "/kaggle/working/dataset/labels/train"
val_image_path = "/kaggle/working/dataset/images/val"
val_label_path = "/kaggle/working/dataset/labels/val"

if not os.path.isdir(train_image_path):
    os.makedirs(train_image_path)
if not os.path.isdir(train_label_path):
    os.makedirs(train_label_path)
if not os.path.isdir(val_image_path):
    os.makedirs(val_image_path)
if not os.path.isdir(val_label_path):
    os.makedirs(val_label_path)

# cycle for train dir
for x in range(count_for_train):
    file_jpg = choice(imgs)
    file_xml = file_jpg[:-4] + ".txt"
    
    shutil.copy(os.path.join(crs_path,file_jpg),os.path.join(train_image_path,file_jpg))
    shutil.copy(os.path.join(crs_path,file_xml),os.path.join(train_label_path,file_xml))
    
    imgs.remove(file_jpg)
    xmls.remove(file_xml)
    
# cycle for test dir
for x in range(count_for_val):
    file_jpg = choice(imgs)
    file_xml = file_jpg[:-4] + ".txt"
    
    shutil.copy(os.path.join(crs_path,file_jpg),os.path.join(val_image_path,file_jpg))
    shutil.copy(os.path.join(crs_path,file_xml),os.path.join(val_label_path,file_xml))
    
    imgs.remove(file_jpg)
    xmls.remove(file_xml)

# rest of files
print("images length - ",len(imgs))
print(imgs,xmls)
for x in imgs:
    file_jpg = x
    file_xml = file_jpg[:-4] + ".txt"
    
    shutil.copy(os.path.join(crs_path,file_jpg),os.path.join(val_image_path,file_jpg))
    shutil.copy(os.path.join(crs_path,file_xml),os.path.join(val_label_path,file_xml))
# shutil.copytree(crs_path,val_path)

# Downloading dataset.yaml file from google cloud for the dataset

In [ ]:
!pip install googledrivedownloader

In [ ]:
from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='1KtlDXpLCfqoklUTdrnVASNbxL_AK9qnx',
                                    dest_path='/kaggle/working/dataset/dataset.yaml')

In [ ]:
with open(r'/kaggle/working/dataset/dataset.yaml', 'r') as file:
    lines = file.readlines()
    print(lines)

# Training the YoloV5 model on custom traffic signs dataset

In [ ]:
!wandb disabled
!python train.py --img 640 --batch 32 --epochs 100 --data /kaggle/working/dataset/dataset.yaml --weights yolov5s.pt --cache --workers 4

# Making Predictions on Validation data

In [ ]:
!ls /kaggle/working/dataset/images/val/

In [ ]:
!python detect.py --source /kaggle/working/dataset/images/val/00358.jpg --weights runs/train/exp3/weights/best.pt

# Displaying results

In [ ]:
!cp runs/train/exp3/weights/best.pt /kaggle/working/

In [ ]:
!ls ./runs/detect/exp

In [ ]:
Image(filename='/kaggle/working/dataset/images/val/00112.jpg', width=416)

In [ ]:
Image(filename='./runs/detect/exp4/00358.jpg', width=416)

In [ ]:
!ls ../